In [16]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt

In [52]:
#opening connection & cursor
conn = psycopg2.connect(
                            host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                            database="mid_term_project",
                            user="lhl_student",
                            password="lhl_student")
cursor = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [95]:

# for i in range(1, 13):
#     if i == 1:
#         test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND WHERE fl_date IN ('2018-01-01','2018-01-02','2018-01-03','2018-01-04','2018-01-05','2018-01-06','2018-01-07','2019-01-01','2019-01-02','2019-01-03','2019-01-04','2019-01-05','2019-01-06','2019-01-07');"
#         df = create_pandas_table(test2)
#     else:
#         test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 100000;"
#         test_pd_table = create_pandas_table(test2)
#         df = pd.concat([df, test_pd_table]) 
test2 = "SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND fl_date IN ('2018-01-01','2018-01-02','2018-01-03','2018-01-04','2018-01-05','2018-01-06','2018-01-07','2019-01-01','2019-01-02','2019-01-03','2019-01-04','2019-01-05','2019-01-06','2019-01-07');"
df = create_pandas_table(test2)

In [97]:
df.shape

(55739, 42)

In [98]:
df.fl_date.unique()

array(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
       '2019-01-05', '2019-01-06', '2019-01-07', '2018-01-01',
       '2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
       '2018-01-06', '2018-01-07'], dtype=object)

In [67]:
passengers = create_pandas_table("SELECT * FROM passengers TABLESAMPLE SYSTEM(10) WHERE year >= 2019 ")

In [68]:
passengers.shape

(48575, 38)

In [69]:
passengers.passengers.max()

66868.0

In [70]:
fuel = create_pandas_table("SELECT * FROM fuel_comsumption WHERE year >= 2019 ")

In [71]:
fuel.shape
fuel.total_gallons

0        1235210.0
1              0.0
2          60821.0
3              0.0
4         410186.0
          ...     
453      3943414.0
454            0.0
455            0.0
456        22895.0
457    162505172.0
Name: total_gallons, Length: 458, dtype: float64

In [72]:
fuel_mean=fuel[['unique_carrier','total_gallons']].groupby(['unique_carrier'],as_index=False).mean()

In [73]:
fuel_mean.rename(columns={'unique_carrier':'op_unique_carrier'},inplace=True)


In [74]:
passengers['capacity'] = passengers['seats'] - passengers['passengers']


In [75]:
passengers.rename(columns={'unique_carrier':'mkt_unique_carrier'},inplace=True)


In [76]:
passengers_mean=passengers[['mkt_unique_carrier','origin','dest','passengers', 'capacity']].groupby(['mkt_unique_carrier','origin','dest', ],as_index=False).mean()

In [77]:
passengers_mean.head()

,mkt_unique_carrier,origin,dest,passengers,capacity
0,04Q,ACK,BED,1.000000,7.000000
1,04Q,ACK,BOS,1.000000,7.000000
2,04Q,ACK,BVY,4.000000,4.000000
3,04Q,ACK,EWR,2.000000,6.000000
4,04Q,ACK,HPN,357.666667,183.666667


In [99]:
df_merged=pd.merge(df,passengers_mean,on=['mkt_unique_carrier', 'origin','dest'], how='inner') 

In [100]:
df_merged.shape

(38221, 44)

In [79]:
df_merged=pd.merge(df_merged,fuel_mean, on=['op_unique_carrier'], how='inner') 

In [80]:
df_merged.shape

(37449, 45)

In [81]:
df_merged.isna().sum()

fl_date                    0
mkt_unique_carrier         0
branded_code_share         0
mkt_carrier                0
mkt_carrier_fl_num         0
op_unique_carrier          0
tail_num                   0
op_carrier_fl_num          0
origin_airport_id          0
origin                     0
origin_city_name           0
dest_airport_id            0
dest                       0
dest_city_name             0
crs_dep_time               0
dep_time                   0
dep_delay                  4
taxi_out                   0
wheels_off                 0
wheels_on                  0
taxi_in                    0
crs_arr_time               0
arr_time                   0
arr_delay                  0
cancelled                  0
cancellation_code      17868
diverted                   0
dup                        0
crs_elapsed_time           0
actual_elapsed_time        0
air_time                   0
flights                    0
distance                   0
carrier_delay              0
weather_delay 

In [84]:
df_merged.dropna(axis=0, subset=['dep_delay'], inplace=True)

In [34]:
# remember to close connection to allow for others to use the bandwidth
cursor.close()
conn.close()

In [85]:
df_merged.passengers.unique()

array([ 7215.        ,  6091.5       ,  8405.        , ...,
       10855.        ,  7247.        ,  6460.33333333])

In [36]:
df.mkt_unique_carrier.unique()

array(['AA', 'NK', 'AS', 'DL', 'UA', 'WN', 'G4', 'HA', 'B6', 'F9'],
      dtype=object)

In [37]:
df.shape

(1179349, 42)

In [86]:
df_merged.shape

(37445, 45)

In [87]:
df_merged.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,passengers,capacity,total_gallons
0,2019-01-01,NK,NK,NK,811,NK,N640NK,811,13204,MCO,...,14.0,0.0,0.0,None,NaN,NaN,None,7215.0,706.0,3.937194e+07
1,2019-01-01,NK,NK,NK,803,NK,N625NK,803,13204,MCO,...,1.0,0.0,66.0,None,NaN,NaN,None,7215.0,706.0,3.937194e+07
2,2019-01-02,NK,NK,NK,811,NK,N646NK,811,13204,MCO,...,15.0,0.0,0.0,None,NaN,NaN,None,7215.0,706.0,3.937194e+07
3,2019-01-03,NK,NK,NK,803,NK,N605NK,803,13204,MCO,...,1.0,0.0,39.0,None,NaN,NaN,None,7215.0,706.0,3.937194e+07
4,2019-01-04,NK,NK,NK,811,NK,N635NK,811,13204,MCO,...,11.0,0.0,0.0,None,NaN,NaN,None,7215.0,706.0,3.937194e+07


In [88]:
df_test = df_merged[['mkt_unique_carrier','origin','dest','dep_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'weather_delay', 'carrier_delay']].groupby(['mkt_unique_carrier','origin','dest', ],as_index=False).mean()

In [89]:
df_merged.drop(columns=['first_dep_time','total_add_gtime','longest_add_gtime', 'no_name', 'dep_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay'], inplace=True)

In [90]:
df_merged = pd.merge(df_merged, df_test, on=['mkt_unique_carrier', 'origin', 'dest'] )

In [69]:
# df_merged = df_merged.drop(columns=['dep_delay_x', 'carrier_delay_x', 'weather_delay_x', 'nas_delay_x', 'security_delay_x',
#        'late_aircraft_delay_x',])

In [91]:
len(df_merged.columns)

41

In [92]:
df_merged

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,passengers,capacity,total_gallons,dep_delay,nas_delay,security_delay,late_aircraft_delay,weather_delay,carrier_delay
0,2019-01-01,NK,NK,NK,811,NK,N640NK,811,13204,MCO,...,1189.0,7215.000000,706.000000,3.937194e+07,47.846154,6.538462,0.0,21.000000,0.0,18.692308
1,2019-01-01,NK,NK,NK,803,NK,N625NK,803,13204,MCO,...,1189.0,7215.000000,706.000000,3.937194e+07,47.846154,6.538462,0.0,21.000000,0.0,18.692308
2,2019-01-02,NK,NK,NK,811,NK,N646NK,811,13204,MCO,...,1189.0,7215.000000,706.000000,3.937194e+07,47.846154,6.538462,0.0,21.000000,0.0,18.692308
3,2019-01-03,NK,NK,NK,803,NK,N605NK,803,13204,MCO,...,1189.0,7215.000000,706.000000,3.937194e+07,47.846154,6.538462,0.0,21.000000,0.0,18.692308
4,2019-01-04,NK,NK,NK,811,NK,N635NK,811,13204,MCO,...,1189.0,7215.000000,706.000000,3.937194e+07,47.846154,6.538462,0.0,21.000000,0.0,18.692308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37440,2018-01-05,HA,HA,HA,152,HA,N475HA,152,13830,OGG,...,121.0,2597.000000,1435.000000,2.239415e+07,22.333333,0.000000,0.0,18.333333,0.0,2.666667
37441,2018-01-06,HA,HA,HA,152,HA,N486HA,152,13830,OGG,...,121.0,2597.000000,1435.000000,2.239415e+07,22.333333,0.000000,0.0,18.333333,0.0,2.666667
37442,2018-01-05,HA,HA,HA,244,HA,N487HA,244,12982,LIH,...,201.0,10855.000000,2713.000000,2.239415e+07,13.000000,1.000000,0.0,12.000000,0.0,3.000000
37443,2018-01-05,HA,HA,HA,35,HA,N583HA,35,14107,PHX,...,2917.0,7247.000000,1232.000000,2.239415e+07,12.000000,0.000000,0.0,0.000000,0.0,21.000000


In [57]:
flights_test = create_pandas_table("SELECT * FROM flights_test WHERE fl_date IN ('2020-01-01','2020-01-02','2020-01-03','2020-01-04','2020-01-05','2020-01-06','2020-01-07')")

In [58]:
flights_test.shape

(150623, 20)

In [59]:
flights_test=pd.merge(flights_test,passengers_mean,on=['mkt_unique_carrier', 'origin','dest'], how='inner') 

In [60]:
flights_test=pd.merge(flights_test,fuel_mean, on=['op_unique_carrier'], how='inner') 

In [61]:
flights_test.shape

(101569, 23)

In [62]:
flights_test.to_csv('flights_test_grouped.csv')

In [93]:
df_merged.to_csv('flights_merged.csv')

In [ ]:
df_merged